In [4]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision

from torch_geometric.datasets import QM9
import torch_geometric.transforms as T
import torch
from torch_geometric.loader import DataLoader
from data_utils import *
import random
import os

from data_utils import create_tensorboard_writer

# disable logging
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = T.Compose([
    SelectQM9TargetProperties(properties=["homo", "lumo"]),
    SelectQM9NodeFeatures(features=["atom_type"]),
    DropQM9Hydrogen(),
    T.ToDevice(device=device)
])

dataset = QM9(root="./data", transform=transform)

train_dataset, _, _ = create_qm9_data_split(dataset=dataset)

sample_index = random.randint(0, len(train_dataset))
print(sample_index)
sample = train_dataset[sample_index]

writer = create_tensorboard_writer(experiment_name="qm9-visualization")

print(f"SMILES: {sample.smiles}")
writer.add_image('Molecules (from SMILES)', smiles_to_image(sample.smiles), dataformats="NCHW")
writer.add_image('Molecules (from Graph)', molecule_graph_data_to_image(sample, includes_h=False), dataformats="NCHW")
writer.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
86095
SMILES: [H]C1=C([H])[C@](C#N)(N([H])[H])C(=O)N1[H]
